In [1]:
import os
import xclim
import xarray as xr
import cftime
import glob

C:\Users\mnew\AppData\Local\anaconda3\envs\CLIMEX_BC_v2\Lib\site-packages\xclim\core\units.py:19: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  import cf_xarray.units


In [2]:
# Define a function to standardize time
def standardize_time(ds):
    """
    Decode and standardize the time variable for a single dataset.
    """
    if 'time' in ds.variables:
        # Get the time attributes
        time_units = ds.time.attrs.get('units', None)
        calendar = ds.time.attrs.get('calendar', 'standard')
        
        # Decode the time variable
        if time_units is not None:
            decoded_times = cftime.num2date(ds.time.values, units=time_units, calendar=calendar)
        
        # Assign back to the dataset
        ds['time'] = ('time', decoded_times)

    return ds

In [3]:
#File to correct 'time' and 'time_bnds'
model_correct_time = xr.open_dataset("O:\\Public\\sharing-4270-CERM\\VLYMI\\Climex DK Domain\\pr\\kba\\pr_EUR-11_CCCma-CanESM2_historical_r1-r1i1p1_OURANOS-CRCM5_kba_1h_1955-2100.nc" , decode_times=False)

In [4]:
# Ensembles of CLIMEX1
string = 'kbi  kbh  kbg  kbf  kbe  kbd  kbc  kbb  kba  kcx  kcw  kcv  kcu  kct  kcs  kcr  kcq  kcp  kco  kcn  kcm  kcl  kck  kcj  kci  kch  kcg  kcf  kce  kcd  kcc  kcb  kca  kbz  kby  kbx  kbw  kbv  kbu  kbt  kbs  kbr  kbq  kbp  kbo  kbn  kbm  kbl  kbk  kbj  '
ensembles = string.split( )
ensembles.sort()

In [5]:
# Paths
input_path = "O:\\Public\\sharing-4270-CERM\\VLYMI\\CLIMEX DK Domain\\pr\\"
output_path = "O:\\Public\\sharing-4270-CERM\\VLYMI\\CLIMEX DK Domain\\pr_daily\\"

In [1]:
%%time
for i, ens in enumerate(ensembles):
    print(ens)
    
    file = glob.glob(input_path + ens + "/pr_*")
    
    print(file)
    
    #Grab ensembler member
    model = xr.open_dataset(file[0], decode_times=False)
    
    #Fix time dimensions
    model['time'] = model_correct_time['time']
    model['time_bnds']= model_correct_time['time_bnds']

    model=standardize_time(model)
    model["time"] = model["time"].indexes.to_datetimeindex()

    #Select time period of interest
    #model = model.sel(time=slice("1981-01-01", "2025-12-31"))
    
    #Compute daily sums 
    model_daily= model['pr'].resample(time='1D').sum()
    print("Finished resampling time.")

    #Convert to mm/day (don't need to multiple by 24 since the data is summed across all hourly timesteps)
    model_daily = model_daily*3600

    #Create output directory
    output_dir = f"{output_path}{ens}"
    
    #Create folder path in directroy
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    #Grab model name string
    match = re.search(r'pr_.*?-CRCM5', file[0])
    
    name = match.group(0)

    #Save final output name
    output_path_sf = os.path.join(output_dir, f"{name}_{ens}_day_1955_2100.nc")

    #Export NetCDF
    model_daily.to_netcdf(output_path_sf)


NameError: name 'ensembles' is not defined